# Simulation Results

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import copy
from matplotlib.backends.backend_pdf import PdfPages

# Do not limit number of rows displayed
pd.set_option('display.max_rows', None)
# constants
TO = 295.0
# benchmark_categories = list(['wolf/18D', 'wolf/19A', 'wolf/19B', 'wolf/19C', 'beem', 'goel', '19/mann', '20/mann'])
array_categories = list(['wolf/18A', 'wolf/19A', 'wolf/19B', 'wolf/19C', '19/mann', '20/mann'])

In [9]:
def clean_b2ir_csv(in_file, btor=False):
  df = pd.read_csv(in_file)
  df = df.rename(columns = {'index': 'base'})
  return df

def clean_btorsim_csv(in_file, btor=False):
  df = pd.read_csv(in_file)
  df = df.rename(columns = {'index': 'base'})
  return df

def clean_seahorn_csv(in_file):
  df = pd.read_csv(in_file)
  df = df.rename(columns = {'index': 'base', 'status': 'sea_status'})
  # set trivial unsat
  df.loc[df['base'] == 'simple_alu.btor', 'sea_status'] = 'unsat'
  df.loc[df['base'] == 'vcegar_QF_BV_itc99_b13_p10.btor2', 'sea_status'] = 'unsat'
  if 'sea_status' in df.columns:
    # identify timeouts
    df['sea_status'] = df['sea_status'].fillna('unknown')
    df['sea_status'] = df['sea_status'].astype(str)
    df.loc[df['sea_status'] == 'unknown', 'seahorn_total'] = 1000
    return df[['base', 'seahorn_total', 'sea_status']]
  return df


## Collect CSVs

### BtorSim


In [10]:
BTORSIM_A_18A_WOLF = "/home/jetafese/hwmc20-mlir/runbench/sims/btorsim_array_18A.csv"
BTORSIM_A_19A_WOLF = "/home/jetafese/hwmc20-mlir/runbench/sims/btorsim_array_19A.csv"
BTORSIM_A_19B_WOLF = "/home/jetafese/hwmc20-mlir/runbench/sims/btorsim_array_19B.csv"
BTORSIM_A_19C_WOLF = "/home/jetafese/hwmc20-mlir/runbench/sims/btorsim_array_19C.csv"
BTORSIM_A_19_MANN = "/home/jetafese/hwmc20-mlir/runbench/sims/btorsim_array_19_mann.csv"
BTORSIM_A_20_MANN = "/home/jetafese/hwmc20-mlir/runbench/sims/btorsim_array_20_mann.csv"

In [11]:
bA20_18A = clean_btorsim_csv(BTORSIM_A_18A_WOLF)
bA20_19A = clean_btorsim_csv(BTORSIM_A_19A_WOLF)
bA20_19B = clean_btorsim_csv(BTORSIM_A_19B_WOLF)
bA20_19C = clean_btorsim_csv(BTORSIM_A_19C_WOLF)
bA20_19mann = clean_btorsim_csv(BTORSIM_A_19_MANN)
bA20_20mann = clean_btorsim_csv(BTORSIM_A_20_MANN)
btorsim_array_results = list([bA20_18A, bA20_19A, bA20_19B, bA20_19C, bA20_19mann, bA20_20mann])

### Btor2MLIR 


In [12]:
BTOR2MLIR_A_18A_WOLF = "/home/jetafese/hwmc20-mlir/runbench/resolve/btor2mlir_array_18A.csv"
BTOR2MLIR_A_19A_WOLF = "/home/jetafese/hwmc20-mlir/runbench/resolve/btor2mlir_array_19A.csv"
BTOR2MLIR_A_19B_WOLF = "/home/jetafese/hwmc20-mlir/runbench/resolve/btor2mlir_array_19B.csv"
BTOR2MLIR_A_19C_WOLF = "/home/jetafese/hwmc20-mlir/runbench/resolve/btor2mlir_array_19C.csv"
BTOR2MLIR_A_19_MANN = "/home/jetafese/hwmc20-mlir/runbench/resolve/btor2mlir_array_19_mann.csv"
BTOR2MLIR_A_20_MANN = "/home/jetafese/hwmc20-mlir/runbench/resolve/btor2mlir_array_20_mann.csv"

In [13]:
birA20_18A = clean_btorsim_csv(BTOR2MLIR_A_18A_WOLF)
birA20_19A = clean_btorsim_csv(BTOR2MLIR_A_19A_WOLF)
birA20_19B = clean_btorsim_csv(BTOR2MLIR_A_19B_WOLF)
birA20_19C = clean_btorsim_csv(BTOR2MLIR_A_19C_WOLF)
birA20_19mann = clean_btorsim_csv(BTOR2MLIR_A_19_MANN)
birA20_20mann = clean_btorsim_csv(BTOR2MLIR_A_20_MANN)
btor2mlir_array_results = list([birA20_18A, birA20_19A, birA20_19B, birA20_19C, birA20_19mann, birA20_20mann])

## Compiled Results

In [14]:
concat_dfs = list()
for (btorsim, b2ir) in zip(btorsim_array_results, btor2mlir_array_results):
  b2ir = b2ir.rename(columns = {'cycles': 'b2ir_cycles'})
  btorsim = btorsim.rename(columns = {'cycles': 'btor_cycles'})
  df = pd.merge(btorsim, b2ir[['base', 'b2ir_cycles']], how='inner', on='base')
  print("btor_cycles: ", round(df.loc[:, 'btor_cycles'].mean()))
  print("b2ir_cycles: ", round(df.loc[:, 'b2ir_cycles'].mean()))
  concat_dfs.append(df)
# concat_dfs

btor_cycles:  41
b2ir_cycles:  162950
btor_cycles:  259
b2ir_cycles:  127363
btor_cycles:  10
b2ir_cycles:  200620
btor_cycles:  15
b2ir_cycles:  64736
btor_cycles:  10
b2ir_cycles:  142050
btor_cycles:  677
b2ir_cycles:  313967


In [15]:
def printTotalStats(df, final, bench):
  # BtorSim
  minBtorSim = round(df.loc[:, 'btor_cycles'].min())
  meanBtorSim = round(df.loc[:, 'btor_cycles'].mean())
  maxBtorSim = round(df.loc[:, 'btor_cycles'].max())
  # B2IR
  minB2IR = round(df.loc[:, 'b2ir_cycles'].min())
  meanB2IR = round(df.loc[:, 'b2ir_cycles'].mean())
  maxB2IR = round(df.loc[:, 'b2ir_cycles'].max())
  # Speedup B2IR
  speedupB2IR = round(meanB2IR/meanBtorSim)

  entry = pd.DataFrame.from_dict({
    "base": [bench],
    # BtorSim
    "btorsim_min":  [minBtorSim],
    "btorsim_mean":  [meanBtorSim],
    "btorsim_max":  [maxBtorSim],
    # B2IR
    "b2ir_min":  [minB2IR],
    "b2ir_mean":  [meanB2IR],
    "b2ir_max":  [maxB2IR],
    # speedup
    "speedup" : [speedupB2IR]
  })

  return pd.concat([final, entry], ignore_index=True)

In [16]:
reportDF = pd.DataFrame(columns=['base', 'btorsim_min', 'btorsim_mean', 'btorsim_max',
                                 'b2ir_min', 'b2ir_mean', 'b2ir_max', 'speedup'])
for (bDF, category) in zip(concat_dfs, array_categories):
  reportDF = printTotalStats (bDF, reportDF, category)

In [17]:
reportDF

,base,btorsim_min,btorsim_mean,btorsim_max,b2ir_min,b2ir_mean,b2ir_max,speedup
0,wolf/18A,10,41,560,92310,162950,229380,3974
1,wolf/19A,50,259,360,92900,127363,184050,492
2,wolf/19B,10,10,10,176530,200620,212200,20062
3,wolf/19C,10,15,40,15470,64736,154680,4316
4,19/mann,10,10,10,52830,142050,317080,14205
5,20/mann,10,677,1010,298260,313967,330370,464


In [18]:
# reportDF.to_latex()
reportDF[['base', 'btorsim_min', 'btorsim_mean', 'btorsim_max', 'b2ir_min', 'b2ir_mean', 'b2ir_max']].to_latex()

'\\begin{tabular}{llllllll}\n\\toprule\n & base & btorsim_min & btorsim_mean & btorsim_max & b2ir_min & b2ir_mean & b2ir_max \\\\\n\\midrule\n0 & wolf/18A & 10 & 41 & 560 & 92310 & 162950 & 229380 \\\\\n1 & wolf/19A & 50 & 259 & 360 & 92900 & 127363 & 184050 \\\\\n2 & wolf/19B & 10 & 10 & 10 & 176530 & 200620 & 212200 \\\\\n3 & wolf/19C & 10 & 15 & 40 & 15470 & 64736 & 154680 \\\\\n4 & 19/mann & 10 & 10 & 10 & 52830 & 142050 & 317080 \\\\\n5 & 20/mann & 10 & 677 & 1010 & 298260 & 313967 & 330370 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [20]:
reportDF[['base', 'btorsim_mean', 'b2ir_mean', 'speedup']]
reportDF[['base', 'btorsim_mean', 'b2ir_mean', 'speedup']].to_latex()

'\\begin{tabular}{lllll}\n\\toprule\n & base & btorsim_mean & b2ir_mean & speedup \\\\\n\\midrule\n0 & wolf/18A & 41 & 162950 & 3974 \\\\\n1 & wolf/19A & 259 & 127363 & 492 \\\\\n2 & wolf/19B & 10 & 200620 & 20062 \\\\\n3 & wolf/19C & 15 & 64736 & 4316 \\\\\n4 & 19/mann & 10 & 142050 & 14205 \\\\\n5 & 20/mann & 677 & 313967 & 464 \\\\\n\\bottomrule\n\\end{tabular}\n'